In [ ]:
from pathlib import Path
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model
from tensorflow.keras.mixed_precision import experimental as mixed_precision
import tensorflow as tf
from random import shuffle
from tqdm import tqdm_notebook
import itertools
import cv2
import shutil

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

In [ ]:
dataset = 'PATH TO DATASET'

In [ ]:
num_images = 10
img_size = 128

In [ ]:
print('\nVideo:')
for split in ['train','val','test']:
    for label in ['real','fake']:
        folders = [f for f in os.listdir(f'{dataset}/{split}/{label}') if len([j for j in os.listdir(f'{dataset}/{split}/{label}/{f}') if 'jpg' in j]) == num_images and '.ipynb' not in f]
        print(f'{split}_{label}: {len(folders)}')

In [ ]:
def Xception(input_shape=(None,None,None,3), lr = 1e-3):
    channel_axis = 3
    img_input = layers.Input(input_shape)
    x = layers.TimeDistributed(layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, name='block1_conv1'))(img_input)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block1_conv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block1_conv1_act'))(x)
    x = layers.TimeDistributed(layers.Conv2D(64, (3, 3), use_bias=False, name='block1_conv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block1_conv2_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block1_conv2_act'))(x)

    residual = layers.TimeDistributed(layers.Conv2D(128, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block2_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block2_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(128, (3, 3), padding='same', use_bias=False, name='block2_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block2_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block2_pool'))(x)
    x = layers.add([x, residual])

    residual = layers.TimeDistributed(layers.Conv2D(256, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.Activation('relu', name='block3_sepconv1_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block3_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block3_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(256, (3, 3), padding='same', use_bias=False, name='block3_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block3_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block3_pool'))(x)
    x = layers.add([x, residual])

    residual = layers.TimeDistributed(layers.Conv2D(728, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.Activation('relu', name='block4_sepconv1_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block4_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block4_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block4_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block4_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='block4_pool'))(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.TimeDistributed(layers.Activation('relu', name=prefix + '_sepconv1_act'))(x)
        x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv1'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name=prefix + '_sepconv1_bn'))(x)
        x = layers.TimeDistributed(layers.Activation('relu', name=prefix + '_sepconv2_act'))(x)
        x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv2'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name=prefix + '_sepconv2_bn'))(x)
        x = layers.TimeDistributed(layers.Activation('relu', name=prefix + '_sepconv3_act'))(x)
        x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name=prefix + '_sepconv3'))(x)
        x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name=prefix + '_sepconv3_bn'))(x)

        x = layers.add([x, residual])

    residual = layers.TimeDistributed(layers.Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False))(x)
    residual = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis))(residual)

    x = layers.TimeDistributed(layers.Activation('relu', name='block13_sepconv1_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(728, (3, 3), padding='same', use_bias=False, name='block13_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block13_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block13_sepconv2_act'))(x)
    x = layers.TimeDistributed(layers.SeparableConv2D(1024, (3, 3), padding='same', use_bias=False, name='block13_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block13_sepconv2_bn'))(x)

    x = layers.TimeDistributed(layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block13_pool'))(x)
    x = layers.add([x, residual])

    x = layers.TimeDistributed(layers.SeparableConv2D(1536, (3, 3), padding='same', use_bias=False, name='block14_sepconv1'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block14_sepconv1_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block14_sepconv1_act'))(x)

    x = layers.TimeDistributed(layers.SeparableConv2D(2048, (3, 3), padding='same', use_bias=False, name='block14_sepconv2'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization(axis=channel_axis, name='block14_sepconv2_bn'))(x)
    x = layers.TimeDistributed(layers.Activation('relu', name='block14_sepconv2_act'))(x)

    x = layers.TimeDistributed(layers.GlobalAveragePooling2D(name='avg_pool'))(x)
    
    model = models.Model(img_input, x, name='xception')
    model.compile(optimizer = Adam(lr), loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

def get_callbacks(modelname, es_patience, rlr_patience, verbose = 1):
    checkpointer = ModelCheckpoint(modelname+'.h5', monitor = 'val_loss', save_best_only = True, verbose = verbose, mode = 'min')
    earlystopper = EarlyStopping(monitor = 'val_loss', patience = es_patience, verbose = verbose, mode = 'min')
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', factor = 1/np.sqrt(10), patience = rlr_patience, cooldown = 1 ,verbose = verbose, mode = 'min')
    return checkpointer, earlystopper, reduceLR


In [ ]:
def generator(l, batch_size):
    gen = iter(itertools.cycle(l))
    while 1:
        yield [next(gen) for _ in range(batch_size)]

def get_input(dataset, split, label, folder):
    images = np.zeros((num_images,img_size,img_size,3), dtype = np.float32)
    imagespath = [f for f in os.listdir(f'{dataset}/{split}/{label}/{folder}') if 'jpg' in f]
    for i in range(num_images):
        img = cv2.resize(plt.imread(f'{dataset}/{split}/{label}/{folder}/{imagespath[i]}'), (img_size, img_size), interpolation=cv2.INTER_CUBIC)/255
        images[i] = img
    return [images]

def video_data_generator(dataset, mode = 'train', batch_size = 8):
    
    if mode == 'test':
        batch_size = batch_size // 4

    assert batch_size % 2 == 0
    
    real_folders = [f for f in os.listdir(f'{dataset}/{mode}/real') if '.ipynb' not in f]
    fake_folders = [f for f in os.listdir(f'{dataset}/{mode}/fake') if '.ipynb' not in f]
    
    shuffle(real_folders)
    shuffle(fake_folders)
    
    real_gen = generator(real_folders, batch_size = batch_size // 2)
    fake_gen = generator(fake_folders, batch_size = batch_size // 2)
    
    while True:

        real_folder_batch, fake_folder_batch = next(real_gen), next(fake_gen)
        batch_input = []
        batch_output = []
        for real_folder, fake_folder in zip(real_folder_batch, fake_folder_batch):
            
            batch_input += get_input(dataset, mode, 'real', real_folder)
            batch_input += get_input(dataset, mode, 'fake', fake_folder)
            batch_output += [0., 1.]
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)

        yield (batch_x, batch_y)

In [ ]:
# Video

model_path = f'models/TDCNN_LSTM' # Save models here
Path(model_path).mkdir(parents=True, exist_ok = True)

epochs = 40
batch_size = 16

train_spe = len(os.listdir(f'{dataset}/train/real'))//batch_size
print(train_spe)
val_spe = len(os.listdir(f'{dataset}/val/real'))//batch_size
print(val_spe)
test_spe = len(os.listdir(f'{dataset}/test/real'))//(batch_size//4)
print(test_spe)

# Train multiple times
for attempt in range(3):
    print(f'Attempt #{attempt}')
    foldername = f'{model_path}/{dataset}_xceptionimagenet_attempt{attempt}'
    Path(foldername).mkdir(parents=True, exist_ok=True)
    modelname = f'{foldername}/{dataset}_xceptionimagenet_{num_images}_{img_size}'

    train_datagen = video_data_generator(dataset,'train', batch_size = batch_size)
    val_datagen = video_data_generator(dataset,'val', batch_size = batch_size)
    test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)

    base_model = Xception()
    base_model.load_weights('imagenet')
    base_model.trainable = False
    
    inputs = layers.Input(shape=(None,None,None,3))
    outputs = base_model(inputs, training=False)
    outputs = layers.Dropout(0.5, name = 'lstm_dropout')(outputs)
    outputs = layers.LSTM(1, dtype='float32', name='LSTM_False')(outputs)
    outputs = (outputs * 0.5) + 0.5
    cnnlstm_model = models.Model(inputs, outputs, name='xception_cnnlstm')
    cnnlstm_model.compile(optimizer = Adam(1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])
    callbacks = get_callbacks(modelname, es_patience = 6, rlr_patience = 4, verbose = 1)
    history = cnnlstm_model.fit(train_datagen, verbose = 2, steps_per_epoch = train_spe, epochs = epochs//2, validation_data = val_datagen, validation_steps = val_spe, callbacks = callbacks)
    best_valloss = callbacks[0].best
    cnnlstm_model = load_model(f'{modelname}.h5')
    test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
    print(cnnlstm_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))

    base_model.trainable = True
    cnnlstm_model.compile(optimizer = Adam(1e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])

    callbacks = get_callbacks(modelname + '_finetuned', es_patience = 7, rlr_patience = 4, verbose = 1)
    callbacks[0].best = best_valloss
    history = cnnlstm_model.fit(train_datagen, verbose = 2, steps_per_epoch = train_spe, epochs = epochs//2, validation_data = val_datagen, validation_steps = val_spe, callbacks = callbacks)
    cnnlstm_model = load_model(f'{modelname}.h5')
    test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
    print(cnnlstm_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))

In [ ]:
# Image

model_path = f'models/CNN' # Save models here
Path(model_path).mkdir(parents=True, exist_ok = True)

epochs = 40
batch_size = 16

train_spe = len(os.listdir(f'{dataset}/train/real'))//batch_size
print(train_spe)
val_spe = len(os.listdir(f'{dataset}/val/real'))//batch_size
print(val_spe)
test_spe = len(os.listdir(f'{dataset}/test/real'))//(batch_size//4)
print(test_spe)

# Train multiple times
for attempt in range(3):
    print(f'Attempt #{attempt}')
    foldername = f'{model_path}/{dataset}_xceptionimagenet_attempt{attempt}'
    Path(foldername).mkdir(parents=True, exist_ok=True)
    modelname = f'{foldername}/{dataset}_xceptionimagenet_{num_images}_{img_size}'

    train_datagen = video_data_generator(dataset,'train', batch_size = batch_size)
    val_datagen = video_data_generator(dataset,'val', batch_size = batch_size)
    test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)

    base_model = Xception()
    base_model.load_weights('imagenet')
    base_model.trainable = False
    
    inputs = layers.Input(shape=(None,None,None,3))
    outputs = base_model(inputs, training=False)
    outputs = layers.TimeDistributed(layers.Dropout(0.5), name = 'globavgpool_dropout')(outputs)
    outputs = layers.TimeDistributed(layers.Dense(1,'sigmoid'),name='predictions')(outputs)
    outputs = layers.GlobalAveragePooling1D(dtype='float32',name='average')(outputs)
    cnn_model = models.Model(inputs, outputs, name='xception_cnn')
    cnn_model.compile(optimizer = Adam(1e-3), loss = 'binary_crossentropy', metrics = ['accuracy'])
    callbacks = get_callbacks(modelname, es_patience = 6, rlr_patience = 4, verbose = 1)
    history = cnn_model.fit(train_datagen, verbose = 2, steps_per_epoch = train_spe, epochs = epochs//2, validation_data = val_datagen, validation_steps = val_spe, callbacks = callbacks)
    best_valloss = callbacks[0].best
    cnn_model = load_model(f'{modelname}.h5')
    test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
    print(cnn_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))

    base_model.trainable = True
    cnn_model.compile(optimizer = Adam(1e-6), loss = 'binary_crossentropy', metrics = ['accuracy'])

    callbacks = get_callbacks(modelname + '_finetuned', es_patience = 7, rlr_patience = 4, verbose = 1)
    callbacks[0].best = best_valloss
    history = cnn_model.fit(train_datagen, verbose = 2, steps_per_epoch = train_spe, epochs = epochs//2, validation_data = val_datagen, validation_steps = val_spe, callbacks = callbacks)
    cnn_model = load_model(f'{modelname}.h5')
    test_datagen = video_data_generator(dataset,'test', batch_size = batch_size)
    print(cnn_model.evaluate(test_datagen,steps=2*test_spe,verbose=0))